
For: NUC - OOP - Assessment 2 <br>
Final version date: 201213


# Index
- Section 00: Preface
- Section 01: Imports
- Section 02: Interfaces.
- Section 03: Customers.
- Section 04: Strategy pattern
- Section 05: Concrete strategy components (with decorators)
- Section 06: Concrete BasicAccount super-type.
- Section 07: Observer pattern.
- Section 08: LoanAccount (using all covered patterns)
- Section 08: Loan prefabs (HomeLoan & PersonalLoan)
- Section 09: Factory pattern (LoanFactory).
- Section 10: Partial composite & collection pattern (LoanContainer).
- Section 11: Extending functionality by reusing componets (CurrentAccount)
- Section 12: Extending functionality by adding new components.
- Section 13: Current account prefabs.
- Section 14: Factory pattern continued.
- Section 15: Combining factory & decorator pattern
- Section 16: Adapter pattern (brief)
- Section 17: Repeating Partial composite & collection pattern (AccountContainer).
- Section 18: Extending functionality once more (SavingsAccount).
- Section 19: Savings account prefabs.
- Section 20: Nested factory pattern.
- Section 21: Putting everything together (CustomerFactory)
- Section 22: Facade pattern (for testing everything)
- Section 23: Singleton pattern (potential 'Bank' object)

---

This notebook was created for the given 'ABCBank' scenario and will contain a demonstration for how OOP (object-oriented programming) and related principles can be applied to make code reusable and easy to extend. Patterns that will be covered (in no particular order):
- Strategy.
- Observer.
- Decorator.
- Factory (abstract).
- Adapter.
- Facade.
- Singleton.
- Composite.

The patterns listed above will be demonstrated while implementing the products specified in the assessment document and there will be some explanation for each section (indexed). Additionally, there will be a number of comments with a simple structure, which I assume will be helpful -- the structure is as follows:
- '# //' (normal comments explaining code).
- '# !!' (commants that point out something extra important).

Docstrings are removed, however, because they made the notebook unnecessarily long without adding much benefit (comments are used instead).

Since the detail level of implementations is supposed to mimic 'Sim-u-duck', most error handling is dropped (though tests should demonstrate that code works). Additionally, some shortcuts have been applied for simplification purposes, due to the lack of permanence, UI or other factors. 

<br>
As a final note, class variables will be protected but none will be private because it is assumed that private access modification will be a burden to the person testing this notebook.


---
## Section 01: Imports.

In [1]:

# // For updates & account locking.
import time

# // For random IDs.
from random import randint

# // For interfaces / abstract classes.
from abc import ABCMeta, abstractmethod

---
## Section 02: Interfaces.

Interfaces are useful for establishing protocol by defining some expected behaviour, which is necessary for most patterns that will be covered in this notebook and follows the 'Program to an interface, not an implementation' principle. In some programming languages, there is special syntax for using interfaces but Python uses metaclassess instead -- they are simply classes with a method set that has to be implemented by inheritors.

<br>
Below is an interface which will be used multiple times throughout this notebook (the purpose is explained in more detail further down) -- it simply requires that an 'update'
method is implemented. 

<br>
<b>Note</b> the naming convention for the interface, it is prefixed with 'I' (for interface) and suffixed with an agent noun (ending with 'er'). This is done because it is a convention in some languages, but more importantly because it might make it easier to read code further down.

In [7]:
class IUpdater(metaclass=ABCMeta):
    @abstractmethod
    def update(self):
        pass

# !! Instantiation of an inheritor that does not implement 
# !! the 'update' method will throw an exception. This 
# !! highlights the importance of the design principle
# !! mentioned in the previous cell; a certain behaviour
# !! is guaranteed.
class Inheritor(IUpdater):
    pass

try:
    i = Inheritor()
except:
    print('crashed')

# // Symbolic del; 'Inheritor' was only defined for demonstration
del Inheritor

crashed


---
## Section 03: Customers.

Before demonstrating any design patterns, a Customer class has to be defined because it will be used for type-hints while implementing some of the products that ABCBank could offer. This class will serve two purposes:
- Contain customer-related data (such as name, age, etc)
- Contain 'containers' that manage products (this will be explained in more detail eventually)

<b>Note</b>; 'Customer' could be broken up into smaller parts with interfaces/abstract classes but this is avoided to keep this notebook from getting needlessly complicated and lengthy.

In [6]:


class Customer(IUpdater):
    def __init__(self, age:int, name:str):
        # // Random for simplicity.
        self.__customer_id = randint(0, 10**5)

        # // Int for simplicity purposes, should actually be
        # // derived from datetime.
        self.__age = age

        self.__name = name

        # // Int for simplicity here as well, should also be
        # // derived from something else (account movement?)
        self.__annual_salary = 0.0

        self.__marital_status = False
        # // Simple string for simplicity.
        self.__education = ""

        # !! As mentioned in the previous cell, this class
        # !! will keep containers of products, which will be
        # !! explained further down in this notebook. For
        # !! now, these will be None and without typehints.
        self.__loan_container = None
        self.__current_account_container = None
        self.__saving_account_container = None

    # !! These methods (until the next comment block)
    # !! will simply be used to access internal data.
    def get_id(self) -> int:
        return self.__customer_id

    def get_age(self) -> int:
        return self.__age

    def get_name(self) -> str:
        return self.__name

    def set_name(self, name) -> str:
        self.__name = name

    def get_annual_salary(self) -> float:
        return self.__annual_salary

    def set_annual_salary(self, amount:float):
        self.__annual_salary = amount

    def get_marital_status(self) -> bool:
        return self.__marital_status

    def set_marital_status(self, status:bool):
        self.__marital_status = status

    def get_education(self) -> str:
        return self.__education

    def set_education(self, description:str):
        self.__education = description

    # !! Notice that there are no typehints
    # !! for the next six methods -- that 
    # !! is because loans and accounts have
    # !! some co-dependency with customers,
    # !! which happen to be defined first.
    def get_loan_container(self):
        return self.__loan_container

    def set_loan_container(self, container):
        self.__loan_container = container

    def get_current_account_container(self):
        return self.__current_account_container

    def set_current_account_container(self, container):
        self.__current_account_container = container

    def get_saving_account_container(self):
        return self.__saving_account_container

    def set_saving_account_container(self, container):
        self.__saving_account_container = container


    # !! Customer is an IUpdater and needs this
    # !! method. To avoid re-declaring this class
    # !! further down, it will implement 'update'
    # !! as it is supposed to, which will be
    # !! explained eventually (short descirption
    # !! is that it will use parts of a composite
    # !! pattern).
    def update(self):
        # // Exception will be raised if this block
        # // is attempted before the implementation
        # // is done.
        try:
            # // These three types will also be IUpdater
            self.get_loan_container().update()
            self.get_current_account_container().update()
            self.get_saving_account_container().update()

        except Exception as e:
            msg = 'used Customer.update too'
            msg += ' early in this notebook'
            raise NotImplementedError(msg)

    # // Making getters and setters easier to work with.
    name = property(get_name, set_name)
    annual_salary = property(get_annual_salary, set_annual_salary)
    marital_status = property(get_marital_status, set_marital_status)
    education = property(get_education, set_education)
    loan_container = property(get_loan_container, set_loan_container)
    current_account_container = property(
        get_current_account_container,
        set_current_account_container
    )
    saving_account_container = property(
        get_saving_account_container,
        set_saving_account_container
    )
    


---
## Section 04: Strategy pattern

To demonstrate the benefits of patterns, principles and concepts of OOP, I want to start with implementing product 2 & 3 (loans), which will be defined in sections 04(here) through 10.

<br>
In many ways, a loan is similar to 'normal' accounts but differs conceptually (at least for the purpose of this demonstration) by having a negative balanace that has to be paid over time. In this notebook, an account will be _composed_ with the following components:

- An 'updater' that applies changes to something (such as balance) over time.
- A 'qualifier' that checks if a customer is eligible for a loan, for instance.
- A locking mechanism that prevents withdrawal if some suspicious activities occur -- or if an account is locked over time.
- A 'depositer' which handles deposits.
- A 'withdrawer' which handles withdrawing.

<br>
The components listed above, at least their specifics and requirements, are likely to <b>change</b> over time and use-case. A 'qualifier', for instance, can have different variations for specific needs and should be interchangable with other similar components within the same 'family'. This is a good place to leverage <b> The strategy pattern </b>, which is defined as 

```
            "The strategy pattern defines a family of algorithms, encapsulates 
            each one, and makes them interchangeable. Strategy lets the algorithm 
            vary independently from clients that use it"

```
Using this pattern, we can <b>identify aspects that will vary and separate them from what stays the same</b>. Additionally, complicated class hierarchies are reduced/removed by <b>favoring composition over inheritance</b>, which has the natural benefit of making code safer and easier to reason with.


<br>
In the next few cells, 'interfaces' will be defined for the components listed above, in addition to the concept of an account.



In [14]:
# // Used for the concept of qualiying something, specifically
# // Customer types, as defined in section 03.
class IQualifier(metaclass=ABCMeta):
    @abstractmethod
    def check_qualifies(self, customer:Customer) -> bool:
        pass

In [15]:
# // Locking mechanism is assumed to only be related
# // to time in this scenario. However, it would be
# // simple to extend functionality, as will be 
# // demonstrated in section 11 and 18.
class ITimeLockChecker(metaclass=ABCMeta):
    @abstractmethod
    def seconds_until_unlock(self) -> float:
        pass

    @abstractmethod
    def check_locked(self) -> bool:
        pass

In [16]:
# // Handles deposits -- this is akin to
# // a filter, as is suggested by the
# // return type-hint of 'deposit(...)'
class IDepositer(metaclass=ABCMeta):
    @abstractmethod
    def deposit(self, amount:float) -> float:
        pass


In [17]:
# !! IWithdrawer (defined further down in this cell) uses an 
# !! optional result of type WithdrawOptional, which is defined
# !! first. It is used as an alternative to exceptions (inspired
# !! by languages such as Rust and Swift) such that this notebook
# !! doesn't require lengthy try-except blocks.
class WithdrawOptional:
    # // Could contain a custom Exception subtype but this
    # // was not done for brevity purposes.
    def __init__(self, err_msg:str, return_amount:float=.0, new_balance:float=.0):
        self.return_amount = return_amount
        self.new_balance = new_balance
        self.err_msg = err_msg

# // IWithdrawer is similar to IDepositer (previous cell) but
# // is intended for withdrawing operations.
class IWithdrawer(metaclass=ABCMeta):
    # !! Notice the 'balance' parameter is optional. The concrete
    # !! implementations of this class will act as a filter that
    # !! is meant to be layered (will play a role when decorator
    # !! pattern is discussed eventually) and having an optional
    # !! parameter will make it possible to make a distinction
    # !! between the outer caller and inner layers. 
    @abstractmethod
    def withdraw(self, amount:float, balance:float=None) -> WithdrawOptional:
        pass

In [18]:
# // General concept of an account that handles subcomponents.
class IAccountHandler(IUpdater, IQualifier, IDepositer, 
                        IWithdrawer, ITimeLockChecker, metaclass=ABCMeta):

    @abstractmethod
    def get_id(self) -> int:
        pass

    @abstractmethod
    def get_balance(self) -> float:
        pass

---
## Section 05: Concrete strategy components (with decorators)

In the cell above, an interface was declared for handling account-related sub-components. The concrete implementation of it (done further down this notebook) will use the 'strategy pattern' by implementing abstract methods in a way that will defer calls to properties that represent strategies. A simplified example:

```
class Strategy(metaclass=ABCMeta):
    @abstractmethod
    def do_task(self) -> int:
        pass

class MyStrategy(Strategy):
    def do_task(self) -> int:
        return 1

class IAccountHandler(Strategy):
    def __init__(self, my_strategy:Strategy):
        self.my_strategy = my_strategy

    # // Deferring
    def do_task(self) -> int:
        return self.my_strategy.do_task()


handler = IAccountHandler(my_strategy=MyStrategy())
print(handler.do_task()) # // Will print '1'

```

With the example given above, a new concrete implementation of 'Strategy' can be created and swapped easily with 'MyStrategy', which gives a significant amount of flexibility. But what if concrete implementations 'MyStrategy' will have multiple variations that are similar? An approach could be to have variants that share code but that could lead to an unmanageable amount of permutations and 'break' the 'single responsibility principle' (one class/function, one purpose). A simpler and more flexible approach is to use another pattern, specifically <b> the decorator pattern </b>, and <b>combine</b> it with <b>the strategy pattern </b>
<br> <br>
The decorator pattern is defined as follows: 
```
        "The Decorator Pattern attaches additional responsibilities 
         to an object dynamically. Decorators provide a flexible 
         alternative to subclassing for extended functionality"
```
Simply put, it layers objects in a concentric manner such that each layer recieves an input, does something with it and passes the potentially processed input to the next layer that is stored within. This motion can be bi-directional, from the inner to outer layer as well.


<br>
In the next few cells, the concrete implementations of relevant interfaces will be defined with the decorator pattern in mind, where components (inner layers, suffixed with 'Component') will act as strategies, while wrappers (suffixed with 'Decorator') will act as decorators.


In [21]:
# // Will default to True.
class TautologyQualifyComponent(IQualifier):
    def check_qualifies(self, customer:Customer) -> bool:
        return True

# // Not full decorator. Simplified for brevity.
# // Checks if a Customer is of a certain min age.
class MinAgeQualifyDecorator(IQualifier):
    # !! Note: _is_ and _has_ a IQualifier.
    def __init__(self, qualifier:IQualifier, min_age:int):
        self._qualifier = qualifier
        self._min_age = min_age

    def check_qualifies(self, customer:Customer) -> bool:
        if customer.get_age() < self._min_age:
            return False
        return self._qualifier.check_qualifies(customer)


# !! Variations of IQualifier implementations such as
# !! qualifications for marital status, income, education
# !! etc are not implemented for the purpose of brevity
# !! (the notebook is very long and it is assumed that
# !! adding more components of this type will only make
# !! it more difficult assess without adding novelty)
# !! If they were to be implemented, it would be very
# !! similar to how MinAgeQualifyDecorator was created.

In [22]:
# // Simple deposit filter.
class BasicDepositComponent(IDepositer):
    def deposit(self, amount:float):
        return amount

In [24]:
# // Does simple 'withdraw' operation unsafely. Can be 
# // regarded as a 'naive' implementation that will be
# // a subject to change in the form of a 'bugfix'.
class BasicWithdrawComponent(IWithdrawer):
    def withdraw(self, amount:float, balance:float=None) -> WithdrawOptional:
        if balance is None:
            return WithdrawOptional(err_msg='No balance')

        # !! Does not account for negative balance.
        remainder = balance - amount
        return WithdrawOptional(
            return_amount=amount,
            new_balance=remainder,
            err_msg=""
        )


# // Decorator for IWithdrawer which is safer
# // than BasicWithdrawComponent (previous class).
class WithdrawWithBoundsDecorator(IWithdrawer):
    
    def __init__(self, withdrawer:IWithdrawer):
        self._withdrawer = withdrawer

    # !! Can be considered a 'bugfix' of BasicWithdrawComponent,
    # !! adds responsibility for safer withdraw operation.
    def withdraw(self, amount:float, balance:float=None) -> WithdrawOptional:
        # // Not intended to be outer layer so balance has to be checked.
        if balance is None:
            return WithdrawOptional(err_msg='No balance')

        if amount < 0:
            amount = 0
        
        # // Drain account -- will alter returned amount.
        remainder = balance - amount
        if remainder < 0:
            return self._withdrawer.withdraw(
                amount=amount-abs(remainder), balance=balance)

        # // Normal withdraw.
        return self._withdrawer.withdraw(
            amount=amount, balance=balance)
    

# // Decorator for drawing a fee while withdrawing.
class WithdrawWithFeeDecorator(IWithdrawer):
    def __init__(self, withdrawer:IWithdrawer, tx_fee_percent:float):
        self._withdrawer = withdrawer
        self._tx_fee_percent = tx_fee_percent

    def withdraw(self, amount:float, balance:float=None) -> WithdrawOptional:
        # // Not intended to be outer layer so balance has to be checked.
        if balance is None:
            return WithdrawOptional(err_msg='No balance')

        option = self._withdrawer.withdraw(amount=amount, balance=balance)

        # // Propagate option if nested IWithdrawer had an err.
        if option.err_msg:
            return option

        # // New option with simple fee calculation for brevity (for 
        # // instance, it doesn't handle positive self.tx_fee_percent).
        return WithdrawOptional(
            return_amount=option.return_amount * (1 - self._tx_fee_percent),
            new_balance=option.new_balance,
            err_msg=''
        )


# // Some testing -- intentionally left here.
w = BasicWithdrawComponent()
w = WithdrawWithBoundsDecorator(withdrawer=w)

# // Note; 20% fee.
w = WithdrawWithFeeDecorator(withdrawer=w, tx_fee_percent=0.2)

# // Note 'amount' is larget than 'balance'.
option = w.withdraw(amount=20, balance=10)
print(f'''
    return amt  : {option.return_amount}
    new balance : {option.new_balance}
    err msg     : {option.err_msg}
''')
# !! Notice that return amount is first clamped to 
# !! 'balance' before a fee was taken (20%).


    return amt  : 8.0
    new balance : 0
    err msg     : 



In [25]:
# // Locking component that defaults to unlocked.
class BaseTimeLockComponent(ITimeLockChecker):
    def seconds_until_unlock(self) -> float:
        return 0

    def check_locked(self) -> bool:
        return False


---
## Section 06: Concrete BasicAccount super-type.

In the cell below, a concrete implementation of IAccountHandler will be defined. There are a few options to keep in mind when choosing an approach onward, for instance:

- Have a shared 'BasicAccount' between all account types and all loan types.
    * Pros: Simle to implement and subclass, design patterns used thus far are very flexible and will allow it, re-using code.
    * Cons: Can arguably break the 'Interface Segragation', 'Single responsebility', and 'favor composition over inheritance' principles. Has a high probability of causing side-effects if this 'BasicAccount' class is changed.
- 'Flat' approach where each product category (small loans, homeloans, current accounts, savings accounts) have each their own implementation of 'IAccountHandler'.
    * Pros: Inverse of the 'cons' of previous approach.
    * Cons: More verbose, 'breaking' the 'don't repeat yourself' principle.
- Some mix of the two approaches above

<br>
In this case, the first approach is selected purely for the purpose of keeping this notebook relatively short. As such all products will be derived from the class below.

In [32]:
# !! Will be the concrete base of all products described
# !! in the specification document.
class BasicAccount(IAccountHandler):

    def __init__(self, 
                 account_id:int, 
                 # // Next 4 params are strategies
                 # // that can be wrapped with
                 # // decorators.
                 qualifier:IQualifier,            
                 timelock_checker:ITimeLockChecker, 
                 depositer:IDepositer,
                 withdrawer:IWithdrawer):

        self._account_id = account_id
        self._balance = 0.0

        # // Strategies (with potential decorators).
        self._qualifier = qualifier
        self._timelock_checker = timelock_checker
        self._depositer = depositer
        self._withdrawer = withdrawer

    def get_id(self) -> int:
        return self._account_id

    def get_balance(self) -> float:
        return self._balance

    # // Defer to qualification checking strategy.
    def check_qualifies(self, customer:Customer) -> bool:
        return self._qualifier.check_qualifies(customer=customer)

    # // Defer to timelocking handling strategy.
    def seconds_until_unlock(self) -> float:
        return self._timelock_checker.seconds_until_unlock()

    # // Defer to timelocking handling strategy.    
    def check_locked(self) -> bool:
        return self._timelock_checker.check_locked()

    # // Defer to depositing strategy.
    def deposit(self, amount:float):
        self._balance += self._depositer.deposit(amount=amount)

    # // Defer to depositing strategy + additional checking.
    # // It is more complicated than the other methods because
    # // IWithdrawer is more complicated in nature.
    def withdraw(self, amount:float, balance:float=None) -> WithdrawOptional:
        # // This is the outer layer of any IWithdrawer and will be
        # // differentiated by overriding 'balance'.
        balance = self.get_balance()

        # // Handle locked account.
        if self.check_locked():
            msg = f'account locked for {self.seconds_until_unlock()} seconds'
            return WithdrawOptional(
                return_amount=0,
                new_balance=self._balance,
                err_msg=msg
            )

        option = self._withdrawer.withdraw(amount=amount, balance=balance)
        # // Propagate if nested IWithdrawer had an err.
        if option.err_msg:
            return WithdrawOptional(
                return_amount=0, 
                new_balance=self._balance,
                err_msg=option.err_msg
            )

        # // Set final balance val.
        self._balance = option.new_balance
        return option


    # !! Note, IUpdater has not been implemented -- this responsebility
    # !! falls on inheritors. It could be a strategy as well but that
    # !! approach would make this notebook unnecessarily lengthy without
    # !! adding much value (strategy pattern is covered with other examples).



---
## Section 07: Observer pattern.

Since products related to loans must conceptually have an approach for keeping track of late downpayments, I think this is a good opportunity to introduce a new pattern named <b>The Observer Pattern</b>, defined as follows:
```
        The observer pattern defines a one-to-many dependency 
        between objects so that when one object changes state, 
        all of its dependents are notified and updated automatically
```

In other words, the intent is to construct a pair of types such that one (subject) has a one-to-many relationship with the other (observers) -- when an event occurs in the subject, all observers are notified.

Below is a rudimentary implementation of the observer pattern.

In [28]:

# // Basic requirements.
class IObserverHandler(metaclass=ABCMeta):
    @abstractmethod
    # // 'object' typehint suggests 'any'.
    def update(self, data:object):
        pass

# // Basic requirements.
class ISubjectHandler(metaclass=ABCMeta):
    @abstractmethod
    def attach(self, observer:IObserverHandler):
        pass

    @abstractmethod
    def detach(self, observer:IObserverHandler):
        pass

    @abstractmethod
    def notify(self):
        pass


In [31]:
# !! Note; not handling memory leaks for brevity purposes.
class DownPaymentObserver(metaclass=ABCMeta):
    def __init__(self):
        self._subject = None

    def set_subject(self, subject:ISubjectHandler):
        self._subject = subject

    def update(self, data:object):
        # !! This does not actually do anything
        # !! besides a printout. It is merely
        # !! a demonstration.
        print('notified of downpayment warning')

class DownPaymentSubject(metaclass=ABCMeta):
    def __init__(self):
        self._observers = set()

    def attach(self, observer:IObserverHandler):
        self._observers.add(observer)

    def detach(self, observer:IObserverHandler):
        self._observers.remove(observer)

    def notify(self):
        for observer in self._observers:
            # // 'data' does not matter in this
            # // demonstration -- just an example.
            data = 0
            observer.update(data)

# // Simple test -- left here intentionally.
subject = DownPaymentSubject()
observer = DownPaymentObserver()

# // Link objects.
observer.set_subject(subject)
subject.attach(observer)

# // Should do a printout.
subject.notify()
del subject, observer
        

notified of downpayment warning


---
## Section 08: LoanAccount (using all covered patterns)

At this point, all necessary components for creating a 'LoanAccount' class are implemented. The class ('LoanAccount', defined in the next cell) will be the superclass of all loan-related products and subclass of 'BasicAccount' -- a small hierarchy such as this is a deviation from the 'favor composition over inheritance' but it is done for the reasons explained in section 06. 

<br>
The integration of the observer pattern into LoanAccount will be simplistic because creating a complete banking system has been out of scope (purpose of the scenario has been to '(...) demonstrate and explain the OOP choices that could help (...)'). As such, an 'observer', specifically 'DownPaymentObserver' will be global in the next cell. The 'subject', in this scenario, will be used in 'LoanAccount.check_notify_late_downpayment'

In [33]:
BANK_OBSERVER = DownPaymentObserver()


class LoanAccount(BasicAccount):
    def __init__(self, 
                 account_id:int,
                 # // The next three params
                 # // will differentiate
                 # // loan prefabs (subclasses)
                 interest_daily_amount:float,
                 loan_amount:int,
                 qualifier:IQualifier):

        # // Setting observer for later use.
        self._subject = DownPaymentSubject()
        self._subject.attach(BANK_OBSERVER)
        BANK_OBSERVER.set_subject(self._subject)

        # // For keeping track of updates.
        self._last_updated = time.time()
        # // Account will be drained daily by this number.
        self._interest_daily_amount = interest_daily_amount

        # // Construct standard withdrawer for this class.
        withdrawer = BasicWithdrawComponent()
        withdrawer = WithdrawWithBoundsDecorator(withdrawer=withdrawer)
        withdrawer = WithdrawWithFeeDecorator(
            # // tx_fee_percent is hardcoded for simplicity.
            withdrawer=withdrawer, tx_fee_percent=0.1)

        super().__init__( 
                 account_id=account_id, 
                 qualifier=qualifier,            
                 timelock_checker=BaseTimeLockComponent(), 
                 depositer=BasicDepositComponent(),
                 withdrawer=withdrawer
        )

        # // Simple drain (bypassing IWithdrawer) for
        # // demonstration purposes.
        self._balance -= loan_amount

    # // The next two methods are simplistic
    # // for brevity purposes.

    def daily_drain(self):
        self._balance -= self._interest_daily_amount

    def check_notify_late_downpayment(self):
        if self.get_balance() < 0:
            self._subject.notify()

    def update(self):
        # // Defining day as 1 second instead of
        # // 60 * 60 * 24 for demonstration purposes
        # // further down this notebook.
        day = 1

        time_remainder = self._last_updated + day - time.time()
        if time_remainder <= 0:
            self.daily_drain()
            self.check_notify_late_downpayment()
            
            self._last_updated = time.time()


In [35]:
# // Demonstrating that LoacAccount works.
loan_amount = 10
interest_daily_amount = 1

loan_account = LoanAccount(
    account_id=1, 
    interest_daily_amount=interest_daily_amount, 
    loan_amount=loan_amount, 
    qualifier=None
)

days = 4
for i in range(days):
    # !! Pay debt on the last day. After this,
    # !! there will not be any more notifications.
    if i == days-1:
        loan_account.deposit(loan_amount + days*interest_daily_amount)

    time.sleep(1)
    loan_account.update()
    print(loan_account.get_balance())

del loan_amount, interest_daily_amount, loan_account, days


notified of downpayment warning
-11.0
notified of downpayment warning
-12.0
notified of downpayment warning
-13.0
0.0


---
## Section 08: Loan prefabs (HomeLoan & PersonalLoan)

In the next cell, a couple of loan prefabs are created. They are purely a named/typed configuration of 'LoanAccount'.

In [37]:
class HomeLoan(LoanAccount):
    def __init__(self, account_id:int, loan_amount:float):

        # // Some interest rate calculation. Per the specification
        # // document, this was not supposed to be realistic.
        interest_daily_amount = loan_amount * 0.1
        min_age_qualification = 30

        qualifier = TautologyQualifyComponent()
        qualifier = MinAgeQualifyDecorator(
            qualifier=qualifier, min_age=min_age_qualification)

        super().__init__( 
                 account_id=account_id,
                 interest_daily_amount=interest_daily_amount,
                 loan_amount=loan_amount,
                 qualifier=qualifier
        )

class PersonalLoan(LoanAccount):
    def __init__(self, account_id:int, loan_amount:float):

        # // Some interest rate calculation. Per the specification
        # // document, this was not supposed to be realistic.
        interest_daily_amount = loan_amount * 0.3
        min_age_qualification = 20

        qualifier = TautologyQualifyComponent()
        qualifier = MinAgeQualifyDecorator(
            qualifier=qualifier, min_age=min_age_qualification)

        super().__init__( 
                 account_id=account_id,
                 interest_daily_amount=interest_daily_amount,
                 loan_amount=loan_amount,
                 qualifier=qualifier
        )


In [40]:
# // Demonstration that code works:

# // Create loans and a customer.
loan_amount = 10**4
home_loan = HomeLoan(account_id=1, loan_amount=loan_amount)
personal_loan = PersonalLoan(account_id=2, loan_amount=loan_amount)
customer = Customer(age=29, name='exam ple')

# // Check if the customer qualifies for these loans.
print('home loan qualifies:', home_loan.check_qualifies(customer))
print('pers. loan qualifies:', personal_loan.check_qualifies(customer))

# // Update to check if a notification is sent with the observer pattern.
print('---\nshould be a notification:')
time.sleep(1)
personal_loan.update()

# // Deposit (not with interest) and update again.
print('---\nshould be a notification:')
personal_loan.deposit(loan_amount)
time.sleep(1)
personal_loan.update()

# // Paid interest _ some arbitrary margin.
# // This should not print a notification.
print('---\nshould _not_ be a notification:')
personal_loan.deposit(10**7)
time.sleep(1)
personal_loan.update()

del loan_amount, home_loan, personal_loan, customer

home loan qualifies: False
pers. loan qualifies: True
---
should be a notification:
notified of downpayment warning
---
should be a notification:
notified of downpayment warning
---
should _not_ be a notification:


---
## Section 09: Factory pattern (LoanFactory).

Thus far, instantiations have been unstructured and that worked fairly well but there would be a benefit in creating objects in a consistent manner. Incidentally, there is a pattern for this, defined as:
```
        "[The abstract factory pattern] Provides an interface 
         for creating families of related or dependent objects 
         without specifying their concrete classes."
```
In other words, this pattern takes the responsibility of instantiating related groups of objects. It is a form of pre-emptive refactoring for creating objects consistently.  

In [41]:
# // A 'helper' for loan factories -- it ensures that menus are
# // created consistently.
class ILoanMenuRetriever(metaclass=ABCMeta):
    @abstractmethod
    def menu(self, new_account_id:int, amount:float) -> dict: # // {str:IAccountHandler}
        pass

# // Interface for loan factories. At this moment, there are only two
# // methods (one for each loan type) but that can be easily extended.
class ILoanFactoryHandler(ILoanMenuRetriever, metaclass=ABCMeta):
    @abstractmethod
    def home_loan(self, account_id:int, amount:float):
        pass

    @abstractmethod
    def personal_loan(self, account_id:int, amount:float):
        pass

# // Concrete loan factory.
class LoanFactory(ILoanFactoryHandler):
    def home_loan(self, account_id:int, amount:float):
        return HomeLoan(account_id=account_id, loan_amount=amount)

    def personal_loan(self, account_id:int, amount:float):
        return PersonalLoan(account_id=account_id, loan_amount=amount)

    # // Consistent menu.
    def menu(self, new_account_id:int, amount:float):
        return {
            'home loan'     : self.home_loan(account_id=new_account_id, amount=amount),
            'personal loan' : self.personal_loan(account_id=new_account_id, amount=amount)
        }

---
## Section 10: Partial composite & collection pattern (LoanContainer).

At this moment, two concrete loan prefabs/types are defined but it would not be difficult to implement more, which raises the necessity of being able to contain their instances with some order. <b>Collection & Composite</b> patterns can be used for this purpose; the former is simply a collection (much like a list), while the latter is defined as:
```
        "[The composite pattern] allows you to compose objects into 
         tree like structures to represent part-whole hierarchies. 
         Composite lets clients treat individual objects and compositions 
         of objects uniformly."
```
For this demonstration, however, only a partial composite pattern will be used. Specifically, the LoanContainer type (defined in the next cell) will be treated as the loans it contains only in the sense that they're all of type IUpdater and can be updated.

<br>
The overall purpose of LoanContainer is that it can be nested in Customer types, as mentioned at the start of this notebook, which are also of type IUpdater. This structure will make it possible to update all account types with a single call (by calling 'update()' on a Customers instance.). Additionally, this approach will be a layer of abstraction between accounts and customers, which is an attempt at the principle that states <b>"Strive for loosely coupled designs between objects that interact".</b>


In [42]:
class LoanContainer(IUpdater):
    # !! Uses strategy pattern (loan factories can be switched).
    def __init__(self, loan_factory:ILoanFactoryHandler):
        self._loan_factory = loan_factory
        self._loans = {}

    # // Simplistic id creation.
    def new_loan_id(self) -> int:
        return randint(0,10**5)

    # !! This retrieves the menu of ILoanFactoryHandler,
    # !! which is also ILoanMenuRetriever.
    def new_loan_options(self, loan_amount:float):
        new_id = self.new_loan_id()
        menu = self._loan_factory.menu(
            new_account_id=new_id,
            amount=loan_amount
        )
        # // Only interested in menu keys (strings).
        return list(menu.keys())

    # // Tries to create a new loan and set it to self._loans.
    def new_loan(self, customer:Customer, choice:str, loan_amount:float) -> int:
        # // Get menu.
        new_id = self.new_loan_id()
        menu = self._loan_factory.menu(
            new_account_id=new_id,
            amount=loan_amount
        )
        # // Note IAccountHandler typehint.
        loan:IAccountHandler = menu.get(choice)

        # // Guard invalid choice.
        if loan is None:
            print('invalid menu choice')
            return -1

        # // Guard unqualified.
        if loan.check_qualifies(customer=customer) == False:
            print('customer does not qualify')
            return -1

        self._loans[new_id] = loan
        return new_id

    def get_loan(self, with_id:int) -> LoanAccount:
        return self._loans.get(with_id)

    def get_all(self): # -> [LoanAccount]
        return list(self._loans.values())

    def del_loan(self, with_id:int):
        # // No in-depth checking for brevity purposes.
        if self._loans.get(with_id):
            del self._loans[with_id]

    def update(self):
        # // All IAccountHandler are IUpdater
        for loan in self._loans.values():
            loan.update() 



In [45]:
# // Small test.

# // Create container & customer.
container = LoanContainer(LoanFactory())
customer = Customer(100, 'some name')

# // Try to get a loan.
loan_amount = 10
choices = container.new_loan_options(loan_amount)
container.new_loan(customer, choices[0], loan_amount)
loan = container.get_all()[0]

# // Update loan through container indirection.
print('amount before update', loan.get_balance())
time.sleep(1)
container.update()
print('amount after update', loan.get_balance())

del container, customer, loan_amount, choices, loan

amount before update -10.0
notified of downpayment warning
amount after update -11.0


---
## Section 11: Extending functionality by reusing componets (CurrentAccount)

In the next cell, a super-type for all 'current accounts' is defined. It is 
very similar to LoanAccount so reusing BasicAccount is straight-forward.

In [48]:
# // Putting together all common pieces for 'current accounts'.
class CurrentAccount(BasicAccount):
    def __init__(self, 
                 account_id:int,
                 tx_fee_percent:float, 
                 daily_balance_drain: float, 
                 qualifier:IQualifier):

        # // Used for periodical account drain (current account fee).
        self._daily_balance_drain = daily_balance_drain
        self._last_drain_time = time.time()

        # // Construct standard withdrawer for this class.
        withdrawer = BasicWithdrawComponent()
        withdrawer = WithdrawWithBoundsDecorator(withdrawer=withdrawer)
        withdrawer = WithdrawWithFeeDecorator(
            withdrawer=withdrawer, tx_fee_percent=tx_fee_percent)

        super().__init__( 
                 account_id=account_id, 
                 qualifier=qualifier,            
                 timelock_checker=BaseTimeLockComponent(), 
                 depositer=BasicDepositComponent(),
                 withdrawer=withdrawer
        )

    def update(self):
        # // Defining day as 1 second instead of
        # // 60 * 60 * 24 for demonstration purposes
        # // further down this notebook.
        day = 1

        time_remainder = self._last_drain_time + day - time.time()
        if time_remainder <= 0:
            self.withdraw(
                amount=self._daily_balance_drain, 
                balance=self.get_balance()
            )
            self._last_drain_time = time.time()
            return





---
## Section 11: Extending functionality by reusing componets (CurrentAccount)

Concrete definitions of IQualifiers have been defined but CurrentAccount types require some additional functionality, such as checking if a customer has a home loan, which is simple due to how components of IAccountHandler are designed with strategy & decorator patterns.


In [49]:

# // Simply checks whether or not a Customer has a certain
# // minimum annual salary.
class SalaryQualifyDecorator(IQualifier):
    def __init__(self, qualifier:IQualifier, min_salary:float):
        self._qualifier = qualifier
        self._min_salary = min_salary

    def check_qualifies(self, customer:Customer) -> bool:
        if customer.get_annual_salary() < self._min_salary:
            return False
        return self._qualifier.check_qualifies(customer)


# // Searches through a LoanContainer that is nested in a Customer
# // to check if any of the loans are of type HomeLoan.
class HasHomeloanQualifyDecorator(IQualifier):
    def __init__(self, qualifier:IQualifier):
        self._qualifier = qualifier

    def check_qualifies(self, customer:Customer) -> bool:
        # // Access all loans.
        loan_container = customer.get_loan_container()
        loans = loan_container.get_all()

        # // Check if any of them is a HomeLoan type.
        for loan in loans:
            if type(loan) is HomeLoan:
                return True
        return False or self._qualifier.check_qualifies(customer)

---
## Section 13: Current account prefabs.

The next cell defines 'current accounts' prefabs/configurations
of type CurrentAccount.

In [50]:
# // Anyone can have this account type.
class BronzeAccount(CurrentAccount):
    def __init__(self, account_id):
        super().__init__(
            account_id=account_id,
            tx_fee_percent=0.3,
            daily_balance_drain=3,
            qualifier=TautologyQualifyComponent()
        )

# // Uses the qualifiers defined in the previous cell.
class SilverAccount(CurrentAccount):
    def __init__(self, account_id):
        qualifier = TautologyQualifyComponent()
        qualifier = SalaryQualifyDecorator(qualifier=qualifier, min_salary=10**3)
        qualifier = HasHomeloanQualifyDecorator(qualifier=qualifier)

        super().__init__(
            account_id=account_id,
            tx_fee_percent=0.2,
            daily_balance_drain=2,
            qualifier=qualifier
        )

# // Identical to SilverAccount, except higher requirement for
# // minimum salary and with better interest rates.
class GoldAccount(CurrentAccount):
    def __init__(self, account_id):
        qualifier = TautologyQualifyComponent()
        qualifier = SalaryQualifyDecorator(qualifier=qualifier, min_salary=10**5)
        qualifier = HasHomeloanQualifyDecorator(qualifier=qualifier)

        super().__init__(
            account_id=account_id,
            tx_fee_percent=0.1,
            daily_balance_drain=1,
            qualifier=qualifier
        )


---
## Section 14: Factory pattern continued.

The next cell is similar to section 09, but defines factories for 'current accounts' instead. It is also a setup for the next section.

In [51]:
# // Again, menu retriever but for accounts (such as current and savings).
# // The previous menu retriever interface was not reused for clarity purposes.
class IAccountMenuRetriever(metaclass=ABCMeta):
    @abstractmethod
    def menu(self, new_account_id:int) -> dict: # // {str:IAccountHandler}
        pass

# // Standard abstract factory interface for CurrentAccount types.
class ICurrentAccountFactoryHandler(IAccountMenuRetriever, metaclass=ABCMeta):
    @abstractmethod
    def bronze(self, account_id:int) -> BronzeAccount:
        pass

    @abstractmethod
    def silver(self, account_id:int) -> SilverAccount:
        pass

    @abstractmethod
    def gold(self, account_id:int) -> GoldAccount:
        pass

# // Concrete implementation of the class above.
class CurrentAccountFactory(ICurrentAccountFactoryHandler):
    def bronze(self, account_id:int) -> BronzeAccount:
        return BronzeAccount(account_id=account_id)

    def silver(self, account_id:int) -> SilverAccount:
        return SilverAccount(account_id=account_id)

    def gold(self, account_id:int) -> GoldAccount:
        return GoldAccount(account_id=account_id)

    def menu(self, new_account_id:int) -> dict: # // {str:IAccountHandler}
        return {
            'bronze': self.bronze(account_id=new_account_id),
            'silver': self.silver(account_id=new_account_id),
            'gold'  : self.gold(account_id=new_account_id)
        }


---
## Section 15: Combining factory & decorator pattern.

To provide further demonstration for the flexibility of the patterns covered thus far, a small 'detour' is made in this notebook (until section 17), or more specifically an example of the <b>'classes should be open for extension, but cloesd for modification'</b> principle. Suppose there is necessity for adding more functionality by adding a logger to accounts, for instance. Instead of backtracking and changing previous code, one could simply leverage the patterns used in the previous sections.

<br>
In the next cell a simple pseudo logger implementation is wrapped around an IAccountHandler (such as BronzeAccount) in a way that would not have any side-effects.


In [52]:
# // Printing instead of logging for brevity purposes.
class AccountLoggerDecorator(IAccountHandler):
    def __init__(self, account_handler:IAccountHandler):
        # !! _is_ and _has_ an IAccountHandler.
        self._account_handler = account_handler

    # !! All the methods below simply logs (prints) a
    # !! line before deferring the method call to the
    # !! nested IAccountHandler.

    def get_id(self) -> int:
        print("Getting account id")
        return self._account_handler.get_id()

    def get_balance(self) -> float:
        print("Getting balance")
        return self._account_handler.get_balance()

    def check_qualifies(self, customer:Customer) -> bool:
        print("Checking qualifications")
        return self._account_handler.check_qualifies(customer=customer)

    def seconds_until_unlock(self) -> float:
        print("Checking how long account is locked")
        return self._account_handler.seconds_until_unlock()

    def check_locked(self) -> bool:
        print("Checking if account is locked")        
        return self._account_handler.check_locked()

    def deposit(self, amount:float):
        print("depositing")
        self._account_handler.deposit(amount=amount)

    def withdraw(self, amount:float, balance:float=None) -> WithdrawOptional:
        print("withdrawing")
        return self._account_handler.withdraw(amount=amount, balance=balance)

    def update(self):
        print('doing update')
        super().update()
        

In [54]:
# !! Concrete ICurrentAccountFactoryHandler implementation which 
# !! adds the extra logging behaviour of IAccountHandler types.
# !! This will not be used anywhere besides the next cell (test)
# !! but it could be interchangeable with CurrentAccountFactory.
class LoggedCurrentAccountFactory(ICurrentAccountFactoryHandler):
    def bronze(self, account_id:int) -> BronzeAccount:
        return AccountLoggerDecorator(BronzeAccount(account_id=account_id))

    def silver(self, account_id:int) -> SilverAccount:
        return AccountLoggerDecorator(SilverAccount(account_id=account_id))

    def gold(self, account_id:int) -> GoldAccount:
        return AccountLoggerDecorator(GoldAccount(account_id=account_id))

    def menu(self, new_account_id:int) -> dict: # // {str:IAccountHandler}
        return {
            'bronze': self.bronze(account_id=new_account_id),
            'silver': self.silver(account_id=new_account_id),
            'gold'  : self.gold(account_id=new_account_id)
        }

In [56]:
f = LoggedCurrentAccountFactory()
acc = f.bronze(0)

print('Logger says:')
acc.deposit(amount=10)
acc.get_balance()
option = acc.withdraw(amount=2)
print('---------')
print(f'''
Option (still works):
    return amt  : {option.return_amount}
    new balance : {option.new_balance}
    err msg     : {option.err_msg}
''')

del f, acc, option

Logger says:
depositing
Getting balance
withdrawing
---------

Option (still works):
    return amt  : 1.4
    new balance : 8.0
    err msg     : 



---
## Section 16: Adapter pattern (brief).

A logging variation of ICurrentAccountFactoryHandler was defined in the previous section but suppose a new logger is created eventually and is not compatible with the previous one. Instead of re-writing the implementation done in the previous section, the 'Adapter pattern' can be used to adapt the two loggers. It (adapter pattern) is similar to other wrapping approaches demonstrated previously but differs mostly in intent -- in the next cell, it will rename a method set.


In [57]:
# // Conceptual logger.
class ILogger(metaclass=ABCMeta):
    @abstractmethod
    def log(self, msg):
        pass

# // Imagined original logger.
class OriginalLogger(ILogger):
    def log(self, msg):
        print(msg)

# // Imagined new logger.
class BetterLogger:
    def better_log(self, msg):
        print(msg)

# // Conforming the new logger to
# // the conceptual logger.
class NewLoggerAdapter(ILogger):
    def __init__(self, new_logger):
        self.new_logger = new_logger

    # // expected 'log()' method defers
    # // the call to 'better_log()'
    def log(self, msg):
        self.new_logger.better_log(msg)

---
## Section 17: Repeating Partial composite & collection pattern (AccountContainer).

The next cell is almost identical to LoanContainer defined in section 10; code was not re-used because some parameters have changed (type-hints as well). This section does not add anything new to the OOP design patterns demonstration, it is merely done for completeion and as setup for other sections.


In [58]:
class AccountContainer(IUpdater):
    # !! Note 'account_factory' param could be either
    # !! CurrentAccountFactory or LoggedCurrentAccountFactory
    def __init__(self, account_factory:IAccountMenuRetriever):
        self._account_factory = account_factory
        self._accounts = {} # // is [IAccountHandler]

    def new_account_id(self) -> int:
        return randint(0,10**5)

    # // Get menu of factory.
    def new_account_options(self) -> list:
        new_id = self.new_account_id()
        menu = self._account_factory.menu(new_account_id=new_id)
        return list(menu.keys())

    # // Similar to new_loan in LoanContainer, it attempts to
    # // create a new account and set it to self.
    def new_account(self, customer:Customer, choice:str) -> int:
        new_id = self.new_account_id()
        menu = self._account_factory.menu(new_account_id=new_id)
        # // Note IAccountHandler typehint.
        new_account:IAccountHandler = menu.get(choice)

        if new_account is None:
            # // Simple error handling for brevity.
            print('invalid menu choice')
            return -1

        # // IAccountHandler is also IQualifier.
        if new_account.check_qualifies(customer=customer) == False:
            print('customer does not qualify')
            return -1

        # // Overwrite is not checked for brevity purposes.
        self._accounts[new_id] = new_account
        return new_id
        
    def get_account(self, with_id:int) -> IAccountHandler:
        return self._accounts.get(with_id)

    def get_all(self) -> list:
        return list(self._accounts.values())

    def del_account(self, with_id:int):
        # // No in-depth checking for brevity purposes.
        if self._accounts.get(with_id):
            del self._accounts[with_id]

    def update(self):
        # // All IAccountHandler are IUpdater
        for account in self._accounts.values():
            account.update()
        

    

In [59]:
# // Define a test for account container as a func because
# // it will be re-used further down.

def test_account_container(accounts_container):
    customer = Customer(age=30, name='example name')
    # // Create account.
    options = accounts_container.new_account_options()
    accounts_container.new_account(customer=customer, choice=options[0])
    account = accounts_container.get_all()[0]

    # // Misc operations below for demonstration.
    account.deposit(10)
    print('first balance:', account.get_balance())

    # // Sleep so account can gain some fees.
    time.sleep(1)

    # // Updating container.
    account.update()
    print('balance after update:',account.get_balance())

    withdraw_amount = 2
    option = account.withdraw(amount=withdraw_amount)
    print('---------')
    print(f'''
    Option: (tried to withdraw {withdraw_amount})
        return amt  : {option.return_amount}
        new balance : {option.new_balance}
        err msg     : {option.err_msg}
    ''')


In [60]:
# // 'balance after update' should  naturaly be less than 'first balance'
# // because this is a curret account (which is periodically drained).
# // Additionally, 'return amt' should be less than 'tried to withdraw x'
# // because of withdrawing fees.

account_factory = CurrentAccountFactory()
account_container = AccountContainer(account_factory=account_factory)
test_account_container(account_container)

del account_factory, account_container

first balance: 10.0
balance after update: 7.0
---------

    Option: (tried to withdraw 2)
        return amt  : 1.4
        new balance : 5.0
        err msg     : 
    


---
## Section 18: Extending functionality once more (SavingsAccount).

This section is for implementing 'savings accounts', it is very similar to the implementations of LoanAccount(section 08&09) and CurrentAccount(section 11&12). BasicAccount is extended once more and new ITimelockCheckers are defined for locking accounts for periods of time.


In [65]:
# // Putting together all common pieces for 'savings accounts'.
class SavingsAccount(BasicAccount):
    def __init__(self, 
                 account_id:int,
                 daily_balance_gain:float, 
                 qualifier:IQualifier,
                 timelock_checker:ITimeLockChecker):

        # // Used for periodical account gain/interest.
        self._daily_balance_gain = daily_balance_gain
        self._last_gain_time = time.time()

        # // Construct standard withdrawer for this class.
        withdrawer = BasicWithdrawComponent()
        withdrawer = WithdrawWithBoundsDecorator(withdrawer=withdrawer)

        super().__init__( 
                 account_id=account_id, 
                 qualifier=qualifier,            
                 timelock_checker=timelock_checker, 
                 depositer=BasicDepositComponent(),
                 withdrawer=withdrawer
        )

    def update(self):
        # // Simple check for whether or not something has been
        # // deposited. This is rudimentary, off course, but
        # // implemented as it is for brevity purposes.
        if self.get_balance() == 0:
            return

        # // Defining day as 1 second instead of
        # // 60 * 60 * 24 for demonstration purposes
        # // further down this notebook.
        day = 1

        time_remainder = self._last_gain_time + day - time.time()
        if time_remainder <= 0:
            self.deposit(amount=self._daily_balance_gain)
            self._last_gain_time = time.time()

In [66]:
# // Strategy for locking based on amount of days.
class DeltaTimeLockComponent(ITimeLockChecker):
    # // Intended for short (day/month based) account locking.
    def __init__(self, lock_days:int):
        lock_seconds = time.time() + (60 * 60 * 24 * lock_days)
        self._unlock_unix_time = lock_seconds

    def seconds_until_unlock(self) -> float:
        return self._unlock_unix_time - time.time()

    def check_locked(self) -> bool:
        return self.seconds_until_unlock() > 0

# // Strategy for locking based on customer age.
class MinAgeTimeLockComponent(ITimeLockChecker):
    # // Intended for y (year based) account locking.
    def __init__(self, current_age:int, unlock_age:int):
        years = unlock_age - current_age
        lock_seconds = time.time() + (60 * 60 * 24 * 365.25 * years)
        self._unlock_unix_time = lock_seconds

    def seconds_until_unlock(self) -> float:
        return self._unlock_unix_time - time.time()

    def check_locked(self) -> bool:
        return self.seconds_until_unlock() > 0

---
## Section 19: Savings account prefabs.

Similar to section 13, where prefabs/configurations for CurrentAccount are done.

In [67]:
class ShortTermSavingsAccount(SavingsAccount):
    def __init__(self, account_id:int):
        super().__init__(
            account_id=account_id,
            daily_balance_gain=1, 
            qualifier=TautologyQualifyComponent(),
            timelock_checker=DeltaTimeLockComponent(lock_days=90)
        )

class LongTermSavingsAccount(SavingsAccount):
    def __init__(self, account_id:int):
        super().__init__(
            account_id=account_id,
            daily_balance_gain=2, 
            qualifier=TautologyQualifyComponent(),
            timelock_checker=DeltaTimeLockComponent(lock_days=365)
        )

class PensionSavingsAccount(SavingsAccount):
    def __init__(self, account_id:int):
        customer_age = 30
        super().__init__(
            account_id=account_id,
            daily_balance_gain=3, 
            qualifier=TautologyQualifyComponent(),
            timelock_checker=MinAgeTimeLockComponent(
                current_age=customer_age, unlock_age=70
            )
        )

---
## Section 20: Nested factory pattern.

For additional flexibility, factory patterns can be nested -- in the next cell a factory for SavingsAccount types is defined, after that (and the test cell) it is used inside another factory for creating consistent instantiation of AccountContainer types.

In [68]:
class ISavingsAccountFactoryHandler(IAccountMenuRetriever, metaclass=ABCMeta):
    @abstractmethod
    def short_term(self, account_id:int) -> ShortTermSavingsAccount:
        pass

    @abstractmethod
    def long_term(self, account_id:int) -> LongTermSavingsAccount:
        pass

    @abstractmethod
    def pension(self, account_id:int) -> PensionSavingsAccount:
        pass

class SavingsAccountFactory(ISavingsAccountFactoryHandler):
    def short_term(self, account_id:int) -> ShortTermSavingsAccount:
        return ShortTermSavingsAccount(account_id=account_id)

    def long_term(self, account_id:int) -> LongTermSavingsAccount:
        return LongTermSavingsAccount(account_id=account_id)

    def pension(self, account_id:int) -> PensionSavingsAccount:
        return PensionSavingsAccount(account_id=account_id)

    def menu(self, new_account_id:int) -> dict: # // {str:IAccountHandler}
        return {
            'short term': self.short_term(account_id=new_account_id),
            'long term' : self.long_term(account_id=new_account_id),
            'pension'   : self.pension(account_id=new_account_id)
        }

In [69]:
# // Note, 'balance after update' should have increased after 'first balance'
# // due to how the savings account accumulates interest for the customer over
# // time. Additionally, there should be an 'err msg' because the account is locked,
# // so the withdrawing attempt failed.

account_factory = SavingsAccountFactory()
account_container = AccountContainer(account_factory=account_factory)
test_account_container(account_container)

del account_factory, account_container

first balance: 10.0
balance after update: 11.0
---------

    Option: (tried to withdraw 2)
        return amt  : 0
        new balance : 11.0
        err msg     : account locked for 7775998.99732995 seconds
    


In [71]:
# !! Interface for creating AccountContainer types. It is 
# !! intended to be used with another factory (nested
# !! inside methods, as seen in AccountContainerFactory
# !! at the bottom of this cell).


class IAccountContainerFactoryHandler(metaclass=ABCMeta):
    @abstractmethod
    def current(self) -> AccountContainer:
        pass
        
    @abstractmethod
    def savings(self) -> AccountContainer:
        pass


class AccountContainerFactory(IAccountContainerFactoryHandler):
    def current(self) -> AccountContainer:
        account_factory = CurrentAccountFactory()
        return AccountContainer(account_factory=account_factory)
        
    def savings(self) -> AccountContainer:
        account_factory = SavingsAccountFactory()
        return AccountContainer(account_factory=account_factory)


In [72]:
# // This test should have similar results to the previous test.

account_container = AccountContainerFactory().savings()
test_account_container(account_container)
del account_container


first balance: 10.0
balance after update: 11.0
---------

    Option: (tried to withdraw 2)
        return amt  : 0
        new balance : 11.0
        err msg     : account locked for 7775998.998837233 seconds
    


---
## Section 21: Putting everything together (CustomerFactory)

At the top of this notebook, the Customer type was defined and it was noted that it required containers for different account types (LoanContainer & AccountContainer). To ensure that everything is created consistently, a factory pattern is used once again because it is appropriate in a context such as this.



In [73]:
class ICustomerFactoryHandler(metaclass=ABCMeta):
    @abstractmethod
    def normal_customer(self, age:int, name:str) -> Customer:
        pass

# // Creates customers with appropriate containers.
class CustomerFactory(ICustomerFactoryHandler):
    def normal_customer(self, age:int, name:str) -> Customer:
        customer = Customer(age=age, name=name)
        
        # // Create & Set current account container.
        current_account_container = AccountContainerFactory().current()
        customer.set_current_account_container(
            container=current_account_container)

        # // Create & Set saving account container.
        saving_account_container = AccountContainerFactory().savings()
        customer.set_saving_account_container(
            container=saving_account_container)

        # // Create & Set loan account container.
        loan_account_container = LoanContainer(LoanFactory())
        customer.set_loan_container(
            container=loan_account_container)

        return customer
        

---
## Section 22: Facade pattern (for testing everything)

It would be useful to have some testing mechanism now that everything is neatly packaged in CustomerFactory. Tests can quickly get complicated and are prone to having code duplication -- this is arguably a good opportunity to introduce the 'Facade' pattern, which simply gives a straight-forward UI that hides away complex code, much like a facade for buildings hides away plumbing, wires, and so on.

In [77]:
# // Nothing is private or protected here on purpose.
# //
# // It is a relatively long class but to summerise:
# //  - adds customers.
# //  - retrieves customers.
# //  - adds current accounts to customers.
# //  - adds loans to customers.
# //  - updates customers and all nested IUpdater types.

class TestingCustomerFacade:
    def __init__(self):
        self.customers = [] # // [Customer]

    # // Add new customers to self.
    def add_customer(self, age:int, name:str):
        customer = CustomerFactory().normal_customer(
            age=age, name=name)
        self.customers.append(customer)

    # // Retrieve customers from self by customer name.
    def get_customer_by_name(self, name:str):
        for customer in self.customers:
            if customer.get_name() == name:
                return customer
        print('customer not found')

    # // Hides complexity for adding current accounts to customers.
    # // Also accepts 'deposit' which will be added to the new account
    # // for testing purposes.
    def add_current_account(self, 
                            customer_name:str,
                            account_choice:str,
                            deposit:float) -> int:
        
        # // Get customer and guard None.
        customer = self.get_customer_by_name(name=customer_name)
        if customer == None:
            print('chould not add current account')
            return -1

        # // Retrieve relevant container.
        container = customer.current_account_container
        account_id = container.new_account(
            customer=customer,
            choice=account_choice
        )

        # // Retrieve relevant accout and guard none.
        account = container.get_account(with_id=account_id)
        if account == None:
            return -1

        # // Deposit and return account id.
        account.deposit(amount=deposit)
        return account_id
        

    # // This is similar to self.add_current_account but
    # // tries to create a new loan instead.
    def add_loan_account(self, 
                         customer_name:str, 
                         account_choice:str,
                         loan_amount:float) -> int:

        # // Retrieve relevant customer.
        customer = self.get_customer_by_name(name=customer_name)
        if customer == None:
            print('chould not add current account')
            return -1
        
        # // Try adding new loan and return its ID.
        loan_id = customer.loan_container.new_loan(
            customer=customer,
            choice=account_choice,
            loan_amount=loan_amount
        )
        return loan_id

    # // Updates a customer and all nested IUpdater types for
    # // a certain amount of time.
    def tick_update_print_balance(self, customer_name:str, seconds:int):
        customer = self.get_customer_by_name(name=customer_name)
        if customer == None:
            print('chould not add current account')
            return
        # // Retrieve all accounts.
        current_accounts = customer.current_account_container.get_all()
        saving_accounts = customer.saving_account_container.get_all()
        loan_accounts = customer.loan_container.get_all()

        # // Simple func for extracting balances from multiple accounts.
        balances = lambda accounts: [x.get_balance() for x in accounts]

        # // Tick and print.
        for _ in range(seconds):
            time.sleep(1)
            customer.update()
            print(f'''\nBalances of..
                current accounts: {balances(current_accounts)}
                saving accounts : {balances(saving_accounts)}
                loan accounts   : {balances(loan_accounts)}
            ''')

            

In [75]:
testing_facade = TestingCustomerFacade()
testing_facade.add_customer(age=30, name="test")

# // Customer tries to get a gold account which requires
# // some minimum annual salary _or_ a home loan.
print("----\nTries to get gold current account. should fail:")
_ = testing_facade.add_current_account(
    customer_name="test", 
    account_choice="gold", 
    deposit=10
)

# // Customer gets a home loan to qualify for a gold current
# // account. This should not be a problem.
print('----\nTries to get homeloan, should be no errors:')
_ = testing_facade.add_loan_account(
    customer_name="test", 
    account_choice="home loan", 
    loan_amount=10**5
)

# // Customer re-applied for a gold account after
# // getting a home loan -- this should work.
print("----\nRe-applies for gold current account"+ 
        "after getting home-loan. should not fail:")
_ = testing_facade.add_current_account(
    customer_name="test", 
    account_choice="gold", 
    deposit=10
)



# // Checking balances between updates to check
# // account draining and demonstrating benefit of 
# // partial composite pattern.
testing_facade.tick_update_print_balance(customer_name="test", seconds=2)

del testing_facade

----
Tries to get gold current account. should fail:
customer does not qualify
----
Tries to get homeloan, should be no errors:
----
Re-applies for gold current accountafter getting home-loan. should not fail:
notified of downpayment warning

Balances of..
                current accounts: [9.0]
                saving accounts : []
                loan accounts   : [-110000.0]
            
notified of downpayment warning

Balances of..
                current accounts: [8.0]
                saving accounts : []
                loan accounts   : [-120000.0]
            


---
## Section 23: Singleton pattern (potential 'Bank' object)

The final pattern demonstrated in this notebook is the 'Singleton', which is essentially used to create one-of-a-kind objects. That is useful for types that are exceptionally large, such as monolithic programs (the potential ABCBank, in this context) or objects that should logically only have one instance (drivers is an example).

This is achieved in Python by using the constructor (__new__ not __init__) to only return a new instance if it is not already created and set as a static variable in the singleton class, as demonstrated in the next cell.

In [78]:
# !! Code from NUC slides.
# !! Note, concurrency is not accounted for in
# !! this example for simplicity purposes.

class Singleton(object):
    # // Static variable, not specific to instances.
    _instance = None
    def __new__(cls):
        # // Simple check if instance of Singleton
        # // already exists.
        if cls._instance is None:
            cls._instance = object.__new__(cls)
        return cls._instance


class Bank(Singleton):
    pass
    # // Method set not included,
    # // it would not add any value
    # // to the example.


In [81]:
# // Demonstrating that two instances
# // of Bank type (child of Singleton)
# // is actually aliasing.

bank1 = Bank()
bank2 = Bank()

# // Same memory id.
print(f'''
    mem id of bank1: {id(bank1)}
    mem id of bank1: {id(bank2)}

    bank1 is bank2?: {bank1 is bank2}
''')

del bank1, bank2


    mem id of bank1: 140532368079504
    mem id of bank1: 140532368079504

    bank1 is bank2?: True

